In [1]:
import sqlite3
import numpy as np
import pandas as pd
import xgboost
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline

In [63]:
conn = sqlite3.connect('../../data/data.db')
# select everything from alcohol where the year is 2021 or less
query = 'SELECT * FROM alcohol WHERE year <= 2021'
original = pd.read_sql_query(query, conn)
df = original
print(df.head())
df.info()

   year    state beverage_type   gallons
0  1970  Alabama       Spirits   3863000
1  1970  Alabama          Wine   1412000
2  1970  Alabama          Beer  33098000
3  1970   Alaska       Spirits    945000
4  1970   Alaska          Wine    470000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7956 entries, 0 to 7955
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           7956 non-null   int64 
 1   state          7956 non-null   object
 2   beverage_type  7956 non-null   object
 3   gallons        7956 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 248.8+ KB


In [64]:
# List of dataframes to test on
data_list = []
state_btype_model = {}
for state in df['state'].unique():
    temp = df[df['state'] == state]
    for btype in temp['beverage_type'].unique():
        state_btype_model[(state, btype)] = []
        data_list.append(temp[temp['beverage_type'] == btype])

# CV Pipeline & Preprocessing

In [65]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

def MLpipe_TSS_MSE_TEST(X, y, preprocessor, ML_algo, param_grid):

    n_splits = 5
    # splits = [0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]
    # diff = 0.05
    splits = [0.5, 0.6, 0.7, 0.8, 0.9]
    diff = 0.10
    
    scores = []
    true = []
    pred = []
    best_models = []
    best_params = []
    
    for i in splits:
        start = int(X.shape[0]*i)
        end = int(X.shape[0]*(i+diff))
        X_other = X[0: start]
        X_test = X[start:end-1]
        y_other = y[0:start]
        y_test = y[start:end-1]
        tscv = TimeSeriesSplit(n_splits=n_splits)
        pipe = make_pipeline(preprocessor, ML_algo)

        # -1 jobs means using all processors
        # mean_squared_error doesn't exists, so we use `neg`
        grid = GridSearchCV(pipe, param_grid = param_grid, scoring='neg_mean_squared_error', cv=tscv, return_train_score = True,
                            n_jobs = -1, verbose=False)
        
        grid.fit(X_other, y_other)
        y_pred = grid.predict(X_test)
        score = mean_squared_error(y_test, y_pred)
        # percet_error = np.mean(np.abs(y_test - y_pred ) / y_pred)
        scores.append(score)
        true.append(y_test)
        pred.append(y_pred)
        best_models.append(grid)
        best_params.append(grid.best_params_)
    return scores, true, pred, best_models, best_params

In [66]:
# Cell to encode data use onehot, minmax, and standard
minmax_ftrs = ['year']

# collect all the encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('minmax', MinMaxScaler(), minmax_ftrs)
    ])

# Models

In [67]:
# test your function with a linear regression model (l1 regularization)
from sklearn.linear_model import Lasso

for df in data_list:
    state = df['state'].unique()[0]
    type = df['beverage_type'].unique()[0]
    y = df['gallons']
    X = df.drop(['gallons', 'state', 'beverage_type'], axis=1)
    param_grid = {
        'lasso__alpha': np.logspace(-2, 2, 21)  # we use logspace since values can go up to inf
    }
    # print("Linear Regression w/ l1")
    l1_scores, l1_true, l1_pred, l1_best_models, l1_best_params = MLpipe_TSS_MSE_TEST(X, y, preprocessor, Lasso(max_iter=1000000), param_grid)
    max_index = np.argmax(np.array(l1_scores))
    state_btype_model[(state, type)].append((l1_scores[max_index], l1_best_models[max_index], "Lasso"))

In [25]:
# Here the prints are just to track how long the model takes, can ignore
# for df in data_list:
#     state = df['state'].unique()[0]
#     type = df['beverage_type'].unique()[0]
#     y = df['gallons']
#     X = df.drop(['gallons', 'state', 'beverage_type'], axis=1)
#     param_grid = {
#         'xgbregressor__n_estimators': [10, 100, 1000],
#         'xgbregressor__max_depth': [5, 6]
#     }
#     xgb_scores, xgb_true, xgb_pred, xgb_best_models, xgb_best_params = MLpipe_TSS_MSE_TEST(X, y, preprocessor, xgboost.XGBRegressor(), param_grid)
#     max_index = np.argmax(np.array(xgb_scores))
#     state_btype_model[(state, type)].append((xgb_scores[max_index], xgb_best_models[max_index], "XGB"))

In [68]:
from sklearn.linear_model import Ridge
for df in data_list:
    state = df['state'].unique()[0]
    type = df['beverage_type'].unique()[0]
    y = df['gallons']
    X = df.drop(['gallons', 'state', 'beverage_type'], axis=1)
    param_grid = {
        'ridge__alpha': np.logspace(-2, 2, 21) # we use logspace since values can go up to inf
    }
    l2_scores, l2_true, l2_pred, l2_best_models, l2_best_params = MLpipe_TSS_MSE_TEST(X, y, preprocessor, Ridge(), param_grid)
    max_index = np.argmax(np.array(l2_scores))
    state_btype_model[(state, type)].append((l2_scores[max_index], l2_best_models[max_index], "Ridge"))

In [69]:
from sklearn.linear_model import ElasticNet
for df in data_list:
    state = df['state'].unique()[0]
    type = df['beverage_type'].unique()[0]
    y = df['gallons']
    X = df.drop(['gallons', 'state', 'beverage_type'], axis=1)
    param_grid = {
        'elasticnet__alpha': 1/np.logspace(-2, 2, 5),
        'elasticnet__l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    }
    en_scores, en_true, en_pred, en_best_models, en_best_params = MLpipe_TSS_MSE_TEST(X, y, preprocessor, ElasticNet(max_iter=5000), param_grid)
    max_index = np.argmax(np.array(en_scores))
    state_btype_model[(state, type)].append((en_scores[max_index], en_best_models[max_index], "Elactic Net"))

In [72]:
import random
def list_argmin(l):
    best = l[0]
    best_ind = 0
    for i, x in enumerate(l):
        if x[0] < best[0]:
            best_ind = i
            best = x
    return best_ind, best

predicted_data = []
for state in original['state'].unique():
    for btype in original['beverage_type'].unique():
        index, (score, best_model, name) = list_argmin(state_btype_model[(state, btype)])
        # (score, best_model, name) = random.choice(state_btype_model[(state, btype)])
        for year in [2022, 2023, 2024]:
            dataframe = pd.DataFrame([year], columns=['year'])
            gallons = best_model.predict(dataframe)[0]
            predicted_data.append((year, state, btype, int(gallons)))

In [73]:
for add in predicted_data:
    year, state, btype, gallons = add
    if state == 'California':
        print(year, state, btype, gallons)


2022 California Spirits 48782104
2023 California Spirits 48782071
2024 California Spirits 48782038
2022 California Wine 132382051
2023 California Wine 133342584
2024 California Wine 134303117
2022 California Beer 692251942
2023 California Beer 695041290
2024 California Beer 697830638


In [74]:
# remove data from 2022-2024 to prevent duplicates
delete_statement = "DELETE FROM alcohol WHERE year >= 2022"
c = conn.cursor()
c.execute(delete_statement)
conn.commit()
insert_statement = "INSERT INTO alcohol (year, state, beverage_type, gallons) VALUES (?, ?, ?, ?)"
c = conn.cursor()
c.executemany(insert_statement, predicted_data)
conn.commit()